In [1]:
import sys
base_dir = "D:/Thesis/Cobra/cobra/"
if base_dir not in sys.path:
    sys.path.append(base_dir)
from os.path import join
import pandas as pd
from utilities import utils
from stats_tools import vis as svis
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.dpi"] = 40
plt.style.use('ggplot')
import importlib
importlib.reload(svis)

<module 'stats_tools.vis' from 'D:/Thesis/Cobra/cobra\\stats_tools\\vis.py'>

In [2]:
fig_dir = join(base_dir, 'figs')
table_dir = join(base_dir, 'data/tables')
dfc = utils.load_scan_csv(join(table_dir, 'neg_pos_clean.csv'))

C:\Users\klein\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  This is separate from the ipykernel package so we can avoid doing imports until


## Get 3d T2 scan pairs, and collect number of slices for those

In [40]:
df_3dt2 = dfc[(dfc.Sequence=='t2') & (dfc.MRAcquisitionType=='3D')]
print(len(df_3dt2), len(df_3dt2.PatientID.unique()))
gb_pat_date = df_3dt2.groupby('PatientID').InstanceCreationDate.nunique()
gb_masked = gb_pat_date[gb_pat_date>1]
pat_scan_pairs = list(gb_masked.index)
print(len(pat_scan_pairs))
df_3dt2_pairs = df_3dt2[df_3dt2.PatientID.isin(pat_scan_pairs)]

7063 5066
213


In [41]:
df_3dt2_pairs_neg = df_3dt2_pairs[df_3dt2_pairs.days_since_test.isna()]
df_3dt2_pairs_nnan = df_3dt2_pairs[~df_3dt2_pairs.days_since_test.isna()]

In [45]:
pat_before = df_3dt2_pairs_nnan[
    df_3dt2_pairs_nnan.days_since_test<=(-30)].PatientID.unique()
pat_post = df_3dt2_pairs_nnan[
    df_3dt2_pairs_nnan.days_since_test>=(-3)].PatientID.unique()
df_3dt2_pre_post = df_3dt2_pairs_nnan[
    df_3dt2_pairs_nnan.PatientID.isin(set(pat_before).intersection(set(pat_post)))]

In [55]:
print(df_3dt2_pre_post.PatientID.unique())
df_3dt2_pre_post.PatientID.nunique()

['42def8d23210b6885b6970854a21ede2' '6e004961c689d4227ef282ccf296c287'
 '11c0a36c5bbfef2b4bd93f631b11a3fc' 'd2743de6e50c912115e532f0194a0e5e']


4

In [110]:
pat_3d_t1t2 = dfc[dfc.PatientID.isin(df_3dt2.PatientID.unique()) &
     (dfc.Sequence=='t1') & (dfc.MRAcquisitionType=='3D')].PatientID.unique()
df_3d_t1t2 = dfc[dfc.PatientID.isin(pat_3d_t1t2)& \
     ((dfc.Sequence=='t1') | (dfc.Sequence=='t2')) & \
          (dfc.MRAcquisitionType=='3D')]

In [111]:
gb = df_3d_t1t2.groupby(['InstanceCreationDate','StudyInstanceUID']).Sequence.nunique()
gb_pairs = gb[gb.values>=2]#
studies_t1t2  = gb_pairs.index.get_level_values(1)
df_3d_t1t2_paired = df_3d_t1t2[df_3d_t1t2.StudyInstanceUID.isin(studies_t1t2)]

In [107]:
df_3d_t1t2_paired.Sequence

0         t2
1         t2
2         t1
4         t2
5         t2
          ..
178452    t1
178453    t2
178456    t1
178457    t1
178458    t2
Name: Sequence, Length: 30950, dtype: object

In [117]:
print(df_3d_t1t2_paired.InstanceCreationDate.nunique(), 'unique dates')

431 unique dates


In [119]:
print(df_3d_t1t2_paired.StudyInstanceUID.nunique(), 'unique studies')

4749 unique studies


In [122]:
print(len(df_3d_t1t2_paired[df_3d_t1t2_paired.Sequence=='t1']), 'T1')
print(len(df_3d_t1t2_paired[df_3d_t1t2_paired.Sequence=='t2']), 'T2')

12350 T1
6375 T2


There are only 4 patients with 3dt2 scan pairs before and after test-->not worth doing domain adaptation

In [130]:
df_3dt2_pre_post.sort_values(by='PatientID')[['PatientID','Rows','Columns']]

,PatientID,Rows,Columns
171958,11c0a36c5bbfef2b4bd93f631b11a3fc,384.0,384.0
171959,11c0a36c5bbfef2b4bd93f631b11a3fc,384.0,384.0
48349,42def8d23210b6885b6970854a21ede2,320.0,320.0
173457,42def8d23210b6885b6970854a21ede2,320.0,320.0
136073,6e004961c689d4227ef282ccf296c287,512.0,512.0
136078,6e004961c689d4227ef282ccf296c287,512.0,512.0
136085,6e004961c689d4227ef282ccf296c287,512.0,512.0
174621,6e004961c689d4227ef282ccf296c287,512.0,512.0
174624,6e004961c689d4227ef282ccf296c287,512.0,512.0
174629,6e004961c689d4227ef282ccf296c287,512.0,512.0


In [126]:
df_3dt2_pre_post.PatientID.unique()

array(['42def8d23210b6885b6970854a21ede2',
       '6e004961c689d4227ef282ccf296c287',
       '11c0a36c5bbfef2b4bd93f631b11a3fc',
       'd2743de6e50c912115e532f0194a0e5e'], dtype=object)

In [124]:
df_3dt2_pre_post.groupby('PatientID').days_since_test.unique()

PatientID
11c0a36c5bbfef2b4bd93f631b11a3fc    [-314.0, 43.0]
42def8d23210b6885b6970854a21ede2    [-118.0, 15.0]
6e004961c689d4227ef282ccf296c287    [-273.0, 70.0]
d2743de6e50c912115e532f0194a0e5e     [-57.0, 13.0]
Name: days_since_test, dtype: object

In [112]:
len(df_3dt2_pairs)

747